<h1>Blanks Report</h1>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.widgets import Slider
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
import warnings
import base64
import io
warnings.filterwarnings('ignore')
%matplotlib inline

<h1 align='center'>Analysis Report</h1>

In [ ]:
# Allows to display all the columns in the dataframe
pd.set_option('display.max_columns', None)

In [ ]:
# Filepath
input_file =  'FILE_PATH_PLACEHOLDER'

In [ ]:
# Read csv and convert it to Pandas dataframe
data = pd.read_csv(input_file)

In [1]:
def export_report(df):
    # Style for tables in HTML export
    css = """
    <style>
    /* General table styling */
    table {
        width: 100%;
        border-collapse: collapse;
        table-layout: auto;
        max-width: 100%;
    }
    th, td {
        padding: 8px;
        text-align: left;
        border: 1px solid #ddd;
        word-wrap: break-word; /* Allows content to wrap within cells */
    }
    
    /* Responsive table container for small screens */
    .scrollable-table {
        display: block;
        width: 100%;
        overflow-x: auto;  /* Enables horizontal scrolling */
        max-width: 100%;
        white-space: nowrap; /* Prevents table from wrapping on small screens */
    }

    /* Responsive adjustments for small screens */
    @media only screen and (max-width: 768px) {
        th, td {
            min-width: 120px; /* Prevents cells from becoming too small */
            font-size: 12px; /* Adjust font size to fit */
        }
    }
    
    /* Print-specific styles */
    @media print {
        /* Hides unnecessary elements during print */
        .navbar, .footer, .sidebar, .button { 
            display: none; 
        }

        /* Ensure tables fit within printable area without clipping */
        .scrollable-table {
            overflow: visible;
        }
        
        body {
            margin: 0;
            padding: 0;
        }
        
        table {
            page-break-inside: auto;
            width: 100%;
        }
        
        /* Prevents rows from breaking across pages */
        tr {
            page-break-inside: avoid;
            page-break-after: auto;
        }
    }
    </style>
    """
    
    # Export DataFrame as HTML with custom style
    df_html = f'<div class="scrollable-table">{df.to_html()}</div>'
    
    # Combine CSS and DataFrame HTML
    full_html = css + df_html
    display(HTML(full_html))

# Example usage


NameError: name 'data' is not defined

In [ ]:
data_rounded = data.copy()
numeric_columns = data.select_dtypes(include='number').columns
data_rounded[numeric_columns].round(2)
styled_data = data_rounded.style.set_properties(**{'font-size': '8pt'}).set_table_styles([{'selector': 'th', 'props': [{'font-size', '8pt'}]}]).format({col: "{:.2f}" for col in numeric_columns}).hide(axis='index')
export_report(styled_data)

,Seq#,Blank,Sample_No,Original QC,Campaign,Drill Hole,Lab,Cert_No,Cert Date,Ag,Au
0,1,334501,334501,Coarse Blank,Tenth Drilling Campaign,JES-24-73,ALS,HE24094468,19/04/24,0.25,0.0025
1,2,334563,334563,Blank,Tenth Drilling Campaign,JES-24-73,ALS,HE24094468,19/04/24,0.25,0.0025
2,3,334636,334636,Blank,Tenth Drilling Campaign,JES-24-73,ALS,HE24094468,19/04/24,0.25,0.0025
3,4,334646,334646,Coarse Blank,Tenth Drilling Campaign,JES-24-74,ALS,HE24094466,17/04/24,0.25,0.0025
4,5,334708,334708,Blank,Tenth Drilling Campaign,JES-24-74,ALS,HE24094466,17/04/24,0.25,0.0025


<h1 align='center'>Statistics</h1>

In [ ]:
def statistics(elements):
  """
    This function returns a dictionary with the statistics of each element

    args:
      elements: a list of elements to be analyzed
    returns:
      a dictionary with the statistics of each element
  """
  elements_dict = {}
  for element in data[elements]:
    count = data[element].count()
    min = data[element].min()
    max = data[element].max()
    mean = data[element].mean()
    std = data[element].std()
    ldl = data['LDL ' + element][0]
    LDLx5 = ldl * 5
    numGreater5Ldl = (data[element] > LDLx5).sum()
    countNumGreater5ldl = (data[element] > LDLx5).count()
    percentPass = ((countNumGreater5ldl-numGreater5Ldl)/countNumGreater5ldl)*100
    percentFail = (100-percentPass)
    elements_dict[element] = [count, min, max, mean, std, ldl, LDLx5, numGreater5Ldl, countNumGreater5ldl, percentPass, percentFail]
  return elements_dict

In [ ]:
stat_data = list(data.columns[9:data.columns.get_loc('LDL ' + data.columns[9])].values.tolist()) # List of the elements that will be used to get the statistics
# Get the statistics of the elements
stats = statistics(stat_data)
# Create a dataframe with the statistics of the elements
stats_df = pd.DataFrame.from_dict(stats, orient='index', columns=['Count', 'Min', 'Max', 'Mean', 'Std Dev', 'LDL', '5xLDL', '#>5LDL', 'Count # > 5LDL', '% Pass', '% Failures'])

stats_df_rounded = stats_df.copy()
numeric_columns = stats_df.select_dtypes(include='number').columns
stats_df_rounded[numeric_columns].round(2)
styled_stats = stats_df_rounded.style.set_properties(**{'font-size': '8pt'}).set_table_styles([{'selector': 'th', 'props': [{'font-size', '8pt'}]}]).format({col: "{:.2f}" for col in numeric_columns})
export_report(styled_stats)

,Count,Min,Max,Mean,Std Dev,LDL,5xLDL,#>5LDL,Count,% Pass,% Failures
Ag,57,0.2500,0.250,0.250000,0.000000,0.500,2.500,0,57,100.0,0.0
Au,57,0.0025,0.015,0.003421,0.002339,0.005,0.025,0,57,100.0,0.0


In [ ]:
# Function to create a downloadable Excel file with two sheets
def create_download_link_multi_sheet(df1, df2, filename="Report.xlsx"):
    # Save the DataFrames to an in-memory Excel file with two sheets
    excel_buffer = io.BytesIO()
    with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
        df1.to_excel(writer, sheet_name='Sheet1', index=False)
        df2.to_excel(writer, sheet_name='Sheet2', index=False)
    
    excel_buffer.seek(0)  # Go to the beginning of the in-memory file
    
    # Convert the Excel file to a base64-encoded string
    b64 = base64.b64encode(excel_buffer.read()).decode()
    
    # Create the HTML button with the download link
    return HTML(f"""
        <a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">
            <button style="font-size:16px; padding:10px; color:white; background-color:green; border:none; border-radius:5px;">
                Download Excel
            </button>
        </a>
    """)

# Display the button
create_download_link_multi_sheet(styled_data, styled_stats)

In [ ]:
data['Cert Date'] = pd.to_datetime(data['Cert Date']) # Convert the Certficate Date column to datetime

<h1 align='center'>Graphs</h1>

In [ ]:
def plot_sequence_number(elements):
    fig = go.Figure()
    ldl_range = 5

    # Create traces for each element
    for element in elements:
        # Scatter plot for markers (blank)
        fig.add_trace(go.Scatter(
            x=data['Seq#'].tolist(),  # Convert Series to list
            y=data[element].tolist(),  # Convert Series to list
            mode='markers',
            name=f'{element} Blank',
            visible=False,
            marker=dict(color='red')
        ))

        # Line for LDL
        fig.add_trace(go.Scatter(
            x=[data['Seq#'].min(), data['Seq#'].max()],
            y=[stats_df.loc[element]['LDL'], stats_df.loc[element]['LDL']],
            mode='lines',
            name=f'{element} Lower Detection Limit',
            visible=False,
            line=dict(dash='dash', color='red')
        ))

        # Line for LDL threshold (default value)
        fig.add_trace(go.Scatter(
            x=[data['Seq#'].min(), data['Seq#'].max()],
            y=[stats_df.loc[element]['LDL'] * ldl_range, stats_df.loc[element]['LDL'] * ldl_range],
            mode='lines',
            name=f'{ldl_range} x LDL Threshold',
            visible=False,
            line=dict(dash='dash', color='blue')
        ))

    # Show the first element by default
    for i in range(3):
        fig.data[i].visible = True

    # Create sliders for each element
    sliders = []
    for element_index, element in enumerate(elements):
        slider = {
            'active': 4,
            'currentvalue': {"prefix": f"LDL x ", "visible": True},
            'pad': {"t": 50},
            'steps': []
        }

        # Slider steps for LDL multiplier
        for i in range(1, 16):
            slider_step = {
                'label': str(i),
                'method': 'restyle',
                'args': [
                    {
                        'y': [
                            data[element].tolist(),  # Markers
                            [stats_df.loc[element]['LDL']] * 2,  # LDL line
                            [stats_df.loc[element]['LDL'] * i] * 2  # LDL threshold
                        ],
                        'name': [
                            f'{element} Blank',
                            f'{element} Lower Detection Limit',
                            f'{i} x LDL Threshold'
                        ]
                    },
                    [element_index * 3, element_index * 3 + 1, element_index * 3 + 2]  # Only update this element
                ]
            }
            slider['steps'].append(slider_step)

        sliders.append(slider)

    # Dropdown buttons for switching elements
    dropdown_buttons = []
    
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)},  # Hide all traces initially
                     {'title': f'Plot for {element}',
                      'xaxis.title.text': f'#Seq',
                      'yaxis.title.text': f'{element}',
                      'sliders': [sliders[i]]}],  # Update the slider for the selected element
            'label': element,
            'method': 'update'
        })

        # Make the current element's traces visible (3 traces per element)
        dropdown_buttons[i]['args'][0]['visible'][i * 3:i * 3 + 3] = [True] * 3

    # Add dropdown menu to update elements
    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'pad': {'r': 10}, 
        }],
        title=f'Plot for {elements[0]}',
        xaxis_title=f'#Seq',
        yaxis_title=f'{elements[0]}',
        width=800,  # Adjust this value based on your needs
        height=600  # Adjust this value based on your needs
    )

    # Set initial slider for the first element
    fig.update_layout(sliders=[sliders[0]])

    fig.show()

In [ ]:
plot_data = stat_data
plot_sequence_number(plot_data)

Dropdown(description='Element:', options=('Ag', 'Au'), value='Ag')

FloatSlider(value=0.0, description='X Min:', max=57.0, readout_format='.3f')

FloatSlider(value=57.0, description='X Max:', max=67.0, min=1.0, readout_format='.3f')

FloatSlider(value=-1.1, description='Y Min:', max=6.1, min=-1.1, readout_format='.3f', step=0.001)

FloatSlider(value=6.1, description='Y Max:', max=6.1, min=-1.1, readout_format='.3f', step=0.001)

Output()

In [ ]:
def plot_dates(elements): 
    fig = go.Figure()
    for element in elements: 
        # Create traces for each element
        fig.add_trace(go.Scatter(x=data['Cert Date'], y=data[element], mode='markers', name='Blank', visible=False, marker=dict(color='red')))
        fig.add_trace(go.Scatter(x=[data['Cert Date'].min(), data['Cert Date'].max()], y=[stats_df.loc[element]['LDL'], stats_df.loc[element]['LDL']], mode='lines', name='Lower Detection Limit', visible=False, line=dict(dash='dash', color='red')))
        fig.add_trace(go.Scatter(x=[data['Cert Date'].min(), data['Cert Date'].max()], y=[stats_df.loc[element]['5xLDL'], stats_df.loc[element]['5xLDL']], mode='lines', name='5x Lower Detection Limit', visible=False, line=dict(dash='dash', color='blue')))

    for i in range(3): 
        # Make he first element visible by default
        fig.data[i].visible = True
    
    # Dropdown button for each element 
    dropdown_buttons = []
    for i, element in enumerate(elements):
        dropdown_buttons.append({
            'args': [{'visible': [False] * len(fig.data)}, 
                     {'title': f'Plot for {element}', 
                      'xaxis.title.text': 'Certificate Date', 
                      'yaxis.title.text': f'{element}'}], 
            'label': element, 
            'method': 'update' 
        })

        dropdown_buttons[i]['args'][0]['visible'][i * 3:i * 3 + 3] = [True] * 3

    fig.update_layout(
        updatemenus=[{
            'buttons': dropdown_buttons, 
            'direction': 'down', 
            'showactive': True, 
            'pad': {'r': 10},
        }], 
        title=f'Plot for {elements[0]}', 
        xaxis_title='Certificate Date',
        yaxis_title=f'{elements[0]}',
        width=800,
        height=600
    )

    fig.show()
    # # Convert the figure to HTML without including Plotly
    # graph_html = fig.to_html(full_html=False, include_plotlyjs=False)
    
    # # Manually include the Plotly CDN link
    # plotly_cdn = '<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>'
    # html_code = f"""
    #      <div style="display: flex; justify-content: center;">
    #         {graph_html}
    #     </div>
    #     {plotly_cdn}
    # """

    # # Display the centered graph
    # display(HTML(html_code))




'\n  for element in data[elements]:\n    plt.figure(figsize=(20, 10))\n    sns.scatterplot(x=data[\'Cert Date\'], y=data[element], color=\'red\', label=\'Blank\')\n    plt.plot([data[\'Cert Date\'].min(), data[\'Cert Date\'].max()], [stats_df.loc[element][\'LDL\'], stats_df.loc[element][\'LDL\']], \'r--\', label="Lower Detection Limit")\n    plt.plot([data[\'Cert Date\'].min(), data[\'Cert Date\'].max()], [stats_df.loc[element][\'5xLDL\'], stats_df.loc[element][\'5xLDL\']], \'b--\', label="5 x LDL Threshold")\n    plt.show()\n'

In [ ]:
plot_dates(plot_data)

Dropdown(description='Element:', options=('Ag', 'Au'), value='Ag')

SelectionRangeSlider(description='Date Range', index=(0, 234), layout=Layout(width='800px'), options=('2024-04…

FloatSlider(value=0.0, description='Y Min:', readout_format='.3f', step=0.001)

FloatSlider(value=0.0, description='Y Max:', readout_format='.3f', step=0.001)

Output()

In [ ]:

# HTML and JS for the print button
html_code = """
    <button style="font-size:16px; padding:10px; color:white; background-color:green; border:none; border-radius:5px;" onclick="window.print()">Print this page</button>
"""

# Display the print button in the notebook, which will be included in the HTML output
display(HTML(html_code))

